In [1]:
import os
import sys
import torch
import torch.nn as nn
import torch.optim as optim

import nibabel as nib
import boto3

/home/warrior/miniconda3/envs/dl/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nb_dir = os.path.split(os.getcwd())[0]
sys.path.append(nb_dir)

In [3]:
def get_s3_data(bucket, key, local_path):
    s3 = boto3.client("s3",
            #aws_access_key_id=os.environ['AWS_ACCESS_KEY_ID'],
            #aws_secret_access_key=os.environ['AWS_SECRET_ACCESS_KEY'],
            )

    with open(local_path, 'wb') as data:
        s3.download_fileobj(bucket, key, data)
    print(f'Saved to: {local_path}')

In [4]:
#s3_PATH = 'apps/62e83227fb855042132724a2/results/model.pt'
s3_PATH = 'apps/62e83227fb855042132724a2/results/model_weights.pt'
LOCAL_PATH = '../local_data/model_weights.pt'

In [5]:
BUCKET = 'test.superbio.ai'
KEY = s3_PATH
get_s3_data(bucket=BUCKET, key=KEY, local_path=LOCAL_PATH)

Saved to: ../local_data/model_weights.pt


In [6]:
%load_ext autoreload
%autoreload 2
from unet.models.uunet_3d import UNet

In [7]:
PATH = '../local_data/model_weights.pt'

In [9]:
#f1 = 64
#filters = [f1, f1 * 2, f1 * 4, f1 * 8, f1 * 16]
model = UNet()
model.load_state_dict(torch.load(PATH, map_location=torch.device('cpu')))
model.eval()
print('')

In [10]:
nii_file  = nib.load('../local_data/test_data/hippo_001.nii.gz')
nii_data = nii_file.get_fdata()
print(type(nii_data))
print('Height, Width, N_images, Channels')
print(nii_data.shape)

<class 'numpy.ndarray'>
Height, Width, N_images, Channels
(32, 32, 32)


In [11]:
x = torch.from_numpy(nii_data)
print(x.shape)
print(x.dtype)

torch.Size([32, 32, 32])
torch.float64


In [12]:
x = x.permute(2, 0, 1)
print(x.shape)
print(x.dtype)

torch.Size([32, 32, 32])
torch.float64


In [13]:
x = torch.unsqueeze(x, dim=0)
print(x.shape)
print(x.dtype)

torch.Size([1, 32, 32, 32])
torch.float64


In [14]:
x = torch.unsqueeze(x, dim=0)
print(x.shape)
print(x.dtype)

torch.Size([1, 1, 32, 32, 32])
torch.float64


In [15]:
y_result = model(x.float())

In [16]:
print(len(y_result))
y_result[0].shape

1


torch.Size([1, 1, 32, 32, 32])

In [17]:
torch.save(y_result[0], '../local_data/prediction/tensor_image.pth')